## twitter demographics 

In [105]:
### Data handling imports
import pandas as pd
import numpy as np
import geopy
### Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import os
from collections import Counter
from collections import OrderedDict
from operator import itemgetter    

import json

# Statistics imports
import scipy, scipy.stats

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')
class color:
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

In [2]:
inputFolder = "../twitter-data-geo-output"
outputFolder = "./twitter-data-demographics-output"

In [7]:
sampleDataList = [x for x in os.listdir(inputFolder) if x.endswith(".json")]
sampleDataList[0]

'20200319-06.json'

In [10]:
sampleDataPath = [os.path.abspath( inputFolder+  "/" + x) for x in sampleDataList]
sampleDataPath[0]

'/scratch/user/diya.li/twitter-action/twitter-data-geo-output/20200319-06.json'

In [12]:
def read_json(filepath):
    with open(filepath, 'r') as file:
        return json.load(file)

In [16]:
sampleJson = read_json(sampleDataPath[0])
sampleTweets = sampleJson[0]
sampleTweets

{'created_at': 'Thu Mar 19 10:59:57 +0000 2020',
 'id': 1240593849694605312,
 'id_str': '1240593849694605312',
 'text': 'current weather in Burlington: overcast clouds, 36°F\n69% humidity, wind 5mph, pressure 1027mb',
 'source': '<a href="http://twitter.com/USACities/cities" rel="nofollow">Cities</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 112034664,
  'id_str': '112034664',
  'name': 'Burlington Weather',
  'screen_name': 'BurlingtonVT',
  'location': 'Burlington, VT',
  'url': None,
  'description': 'Weather updates, forecast, warnings and information for Burlington, VT. Sources: http://OpenWeatherMap.org, NOAA, USGS.',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 120,
  'friends_count': 1,
  'listed_count': 14,
  'favourites_count': 0,
  'statuses_count': 37409,
  'created_

In [42]:
# I think we should filter those weather account or bot account first, I should print all the screen name first
screenNameList = []
geoPointList = []
coordinatesList = []
for j in sampleDataPath:
    sampleJson = read_json(j)
    for i in sampleJson:
        screenNameList.append(i['user']['name'])
        geoPointList.append(i['geo']['type'])
        coordinatesList.append(i['coordinates']['type'])

290007
{'Point'}


In [43]:
print(len(screenNameList))
print(set(geoPointList))
print(set(coordinatesList))

290007
{'Point'}
{'Point'}


In [26]:
nameCounter = Counter(screenNameList)

In [ ]:
frequentDict = OrderedDict()

In [39]:
mostFrequent =  OrderedDict([(x,y) for x, y in sorted(nameCounter.items(), key = itemgetter(1), reverse = True) if y > 10])

In [40]:
len(mostFrequent)

3979

# I think I still need to convert those json files into dataframe

In [44]:
# select the useful keys
sampleTweets.keys()

dict_keys(['created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'])

In [45]:
sampleTweets['user'].keys()

dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'url', 'description', 'translator_type', 'protected', 'verified', 'followers_count', 'friends_count', 'listed_count', 'favourites_count', 'statuses_count', 'created_at', 'utc_offset', 'time_zone', 'geo_enabled', 'lang', 'contributors_enabled', 'is_translator', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'profile_image_url', 'profile_image_url_https', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications'])

In [51]:
sampleDataFrame = pd.DataFrame(columns = rawColumnsName)

In [53]:
sampleDataFrame

,created_at,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms,id,id_str,name,screen_name,location,url,description,translator_type,protected,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,utc_offset,time_zone,geo_enabled,lang,contributors_enabled,is_translator,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,profile_image_url,profile_image_url_https,default_profile,default_profile_image,following,follow_request_sent,notifications


In [109]:
def cleanSampleUser(userJson):
    # clean userJsonn
    keys = userJson.keys()
    for k in list(keys):
        # delete all profile key
        if k.startswith("profile"):
            del userJson[k]
        else:
            userJson["user_" + k] = userJson.pop(k)
    userJson.pop('translator_type', None)
    userJson.pop('is_translator', None)
    userJson.pop('notifications', None)
    userJson.pop('following', None)
    userJson.pop('follow_request_sent', None)
    userJson.pop('id', None)
    
    return userJson


def cleanSamplePlace(placeJson):
    print(placeJson)
    # clean place json
    keys = placeJson.keys()
    for k in list(keys):
        placeJson['place_' + k] = placeJson.pop(k)
    placeJson.pop('url', None)
    
    return placeJson


def cleanSampleTweets(samplePath):
    # read
    output = []
    sampleJsons = read_json(samplePath)
    
    for index, sampleJson in enumerate(sampleJsons):
        userJson = sampleJson['user']
        userJson = cleanSampleUser(userJson)
        sampleJson.pop('user', None)
        #placeJson = sampleJson['place']
        #placeJson = cleanSamplePlace(placeJson)
        #sampleJson.pop('place', None)

        keys = sampleJson.keys()
        for k in list(keys):
            # delete all reply key
            if k.startswith('in_reply'):
                del sampleJson[k]

        # clean sample json
        if sampleJson['coordinates'] is None:
            # the coordinates is differ from the geo tag
            sampleJson['coordinates'] = [sampleJson['geo']['coordinates'][1], sampleJson['geo']['coordinates'][0]]
        else:
            sampleJson['coordinates'] = sampleJson['coordinates']['coordinates']
            
        userJson.pop('truncated', None)
        userJson.pop('is_translator', None)
        userJson.pop('id', None)
        userJson.pop('contributors', None)
        userJson.pop('geo', None)
        userJson.pop('id', None)
        sampleJson.update(userJson)
        #sampleJson.update(placeJson)
        output.append(sampleJson)

    return output

In [110]:
cleaned = []
cleanedSet = {}
for sp in sampleDataPath:
    cleaned += cleanSampleTweets(sp)

for index, i in enumerate(cleaned):
    cleanedSet[str(index)] = i
del cleaned

In [116]:
with open(outputFolder + "/" + "cleanedSet.json", "w") as file:
    json.dump(cleanedSet, file)